In [ ]:
## Considerar computador ryzen 7 5600x, 16 gb de ram e rx 3080 de 10gb.
## Executado no VSCode com python 3.12.4

## Exercício 1 - Função de Rosenbrock
## Avaliar para diferentes tamanhos de D

A formulação da função é dada por:

$$
f(x) = \sum_{i=1}^{n-1} \left[ 100(x_{i+1} - x_i^2)^2 + (1 - x_i)^2 \right]
$$


In [10]:
import numpy as np
import time
from scipy.optimize import minimize

# Combined Rosenbrock function
def rosenbrock(x):
    fval = sum(100.0 * (x[1:] - x[:-1]**2.0)**2.0 + (1 - x[:-1])**2.0)

    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    grad = np.zeros_like(x)
    grad[1:-1] = 200 * (xm - xm_m1**2) - 400 * (xm_p1 - xm**2) * xm - 2 * (1 - xm)
    grad[0] = -400 * x[0] * (x[1] - x[0]**2) - 2 * (1 - x[0])
    grad[-1] = 200 * (x[-1] - x[-2]**2)

    n = len(x)
    H = np.zeros((n, n))
    for i in range(n):
        if i > 0:
            H[i, i - 1] = -400 * x[i - 1]
        if i < n - 1:
            H[i, i] = 1200 * x[i]**2 - 400 * x[i + 1] + 2
            H[i, i + 1] = -400 * x[i]
        else:
            H[i, i] = 200
    for i in range(n - 1):
        H[i + 1, i] = H[i, i + 1]

    return fval, grad, H

# Número de dimensões
n_dim = 300

# Ponto inicial aleatório entre -5 e 5 com 1 casa decimal
x0 = np.round(np.random.uniform(-5, 5, n_dim), 1)

# Lista de métodos para comparar
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: rosenbrock(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
}

# Executa as otimizações e armazena resultados
results = {}

for name, opts in methods.items():
    start_time = time.time()
    res = minimize(lambda x: rosenbrock(x)[0], x0, **opts, options={"disp": False})
    elapsed_time = (time.time() - start_time) * 1000  # em ms
    results[name] = {
        "x0": x0,
        "x*": res.x,
        "fval": res.fun,
        "nfev": res.nfev,
        "njev": res.get("njev", None),
        "nhev": res.get("nhev", None),
        "time": elapsed_time,
        "success": res.success
    }

# Apresenta os resultados com alinhamento
print("\n==== Rosenbrock Results ====\n")
header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
print(header)
print("-" * len(header))

for method, data in results.items():
    print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
          f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
          f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
          f"{data['time']:10.2f} | {str(data['success']):>8}")


==== Rosenbrock Results ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |   1.0413e+06 |  60000 |    N/A |    N/A |   49515.99 |    False
BFGS                 |   3.7309e-14 |   1599 |   1599 |    N/A |    3269.26 |     True
Newton-CG            |   2.8621e+02 |    807 |    807 |    794 |    1004.08 |     True
trust-ncg            |   8.7857e-10 |  14414 |  14401 |  14400 |   19030.95 |     True
trust-krylov         |   3.9944e-09 |   5012 |   5012 |   5006 |    9794.95 |     True
trust-exact          |   3.5804e-09 |   4803 |   4785 |   4803 |   22530.29 |     True


In [ ]:
## teste para 5 dimensões

## Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
## --------------------------------------------------------------------------------------
## Nelder-Mead          |   6.6175e-05 |    243 |    N/A |    N/A |       9.51 |     True
## BFGS                 |   4.0131e-13 |     30 |     30 |    N/A |       3.02 |     True
## Newton-CG            |   1.4917e-02 |   1003 |   1003 |   1000 |     140.99 |    False
## trust-ncg            |   3.3223e-04 |   1001 |    848 |    847 |     120.79 |    False
## trust-krylov         |   2.7387e-09 |    569 |    569 |    561 |     342.38 |     True
## trust-exact          |   8.2289e-10 |    655 |    645 |    655 |     279.26 |     True

## teste para 2 dimensões

## (1, 0.3)

## Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
## --------------------------------------------------------------------------------------
## Nelder-Mead          |   2.0953e-10 |    113 |    N/A |    N/A |       4.00 |     True
## BFGS                 |   2.0153e-13 |     22 |     22 |    N/A |       2.00 |     True
## Newton-CG            |   1.0362e-14 |     24 |     24 |     20 |       3.00 |     True
## trust-ncg            |   9.8517e-09 |     15 |     13 |     12 |       2.00 |     True
## trust-krylov         |   9.0262e-09 |     16 |     16 |     13 |       8.00 |     True
## trust-exact          |   0.0000e+00 |      2 |      2 |      2 |       0.00 |     True

## teste para 10 dimensões

## [ 1.4 -4.7  4.8  4.1 -2.1 -2.1 -5.  -3.4  4.8  3.2]

## Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
## --------------------------------------------------------------------------------------
## Nelder-Mead          |   5.4648e+01 |   2000 |    N/A |    N/A |      85.58 |    False
## BFGS                 |   8.6106e-14 |    160 |    160 |    N/A |      20.13 |     True
## Newton-CG            |   2.1864e-02 |   2028 |   2028 |   2000 |     305.77 |    False
## trust-ncg            |   1.0971e-05 |   2001 |   1947 |   1946 |     283.19 |    False
## trust-krylov         |   3.8616e-09 |    813 |    813 |    800 |     519.17 |     True
## trust-exact          |   2.8287e-09 |    851 |    840 |    851 |     370.64 |     True

## teste para 20 dimensões

## ==== Rosenbrock Results ====

## [-2.5, -9.5, 6.8, -6.9, -6.9, -8.7, 7.4, 5.4, -9.1, 1.6, -0.6, -7.4, -6.5, 3.2, 9.7, -9.3, -9.2, -4.2, 8.4, -6.2]

## Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
## --------------------------------------------------------------------------------------
## Nelder-Mead          |   1.0666e+05 |   4000 |    N/A |    N/A |     217.48 |    False
## BFGS                 |   4.9449e-15 |    323 |    323 |    N/A |      47.59 |     True
## Newton-CG            |   4.5052e-03 |   4030 |   4030 |   4000 |     754.54 |    False
## trust-ncg            |   1.4480e-09 |   3985 |   3973 |   3972 |     711.93 |     True
## trust-krylov         |   3.1793e-09 |   1242 |   1242 |   1233 |     822.85 |     True
## trust-exact          |   2.7978e-09 |   1053 |   1038 |   1053 |     633.66 |     True

## teste para 300 dimensões (essas achei melhor gerar aleatoriamente entre -5 e 5)

## ==== Rosenbrock Results ====

## Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
## --------------------------------------------------------------------------------------
## Nelder-Mead          |   1.0570e+06 |  60000 |    N/A |    N/A |   48800.62 |    False
## BFGS                 |   3.0510e-14 |   2387 |   2387 |    N/A |    4770.42 |     True
## Newton-CG            |   2.4771e-04 |  15866 |  15866 |  15769 |   19983.88 |     True
## trust-ncg            |   8.9459e-10 |   8125 |   8116 |   8115 |   11278.43 |     True
## trust-krylov         |   3.9197e-09 |   3103 |   3103 |   3096 |    6082.07 |     True
## trust-exact          |   3.6826e-09 |   3237 |   3224 |   3237 |   11007.05 |     True


## teste para 300 dimensões (2)
## ==== Rosenbrock Results ====

## Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
## --------------------------------------------------------------------------------------
## Nelder-Mead          |   1.0413e+06 |  60000 |    N/A |    N/A |   49515.99 |    False
## BFGS                 |   3.7309e-14 |   1599 |   1599 |    N/A |    3269.26 |     True
## Newton-CG            |   2.8621e+02 |    807 |    807 |    794 |    1004.08 |     True
## trust-ncg            |   8.7857e-10 |  14414 |  14401 |  14400 |   19030.95 |     True
## trust-krylov         |   3.9944e-09 |   5012 |   5012 |   5006 |    9794.95 |     True
## trust-exact          |   3.5804e-09 |   4803 |   4785 |   4803 |   22530.29 |     True

## Exercício 2- Função 114 - Scahffer 3

## Testar para diferentes pontos iniciais.

$$
f(x, y) = 0.5 + \frac{\sin^2(x^2 - y^2) - 0.5}{\left[1 + 0.001(x^2 + y^2)\right]^2}
$$

 


In [16]:
import numpy as np
import time
from scipy.optimize import minimize

# Função Schaffer N.3
def schaffer_n3(x):
    x1, x2 = x[0], x[1]
    num = np.sin(x1**2 - x2**2)**2 - 0.5
    denom = (1 + 0.001 * (x1**2 + x2**2))**2
    fval = 0.5 + num / denom

    # Gradiente (calculado simbolicamente)
    dx = (
        2 * np.sin(x1**2 - x2**2) * np.cos(x1**2 - x2**2) * 2 * x1 * denom
        - num * 0.002 * x1 * (1 + 0.001 * (x1**2 + x2**2))
    ) / denom**2

    dy = (
        -2 * np.sin(x1**2 - x2**2) * np.cos(x1**2 - x2**2) * 2 * x2 * denom
        - num * 0.002 * x2 * (1 + 0.001 * (x1**2 + x2**2))
    ) / denom**2

    grad = np.array([dx, dy])

    return fval, grad

# Ponto inicial (modifique aqui)
x0 = np.array([7.8, -7.8])

# Lista de métodos para comparação
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: schaffer_n3(x)[1]},
    "CG": {"method": "CG", "jac": lambda x: schaffer_n3(x)[1]},
    "Powell": {"method": "Powell"},
    "L-BFGS-B": {"method": "L-BFGS-B", "jac": lambda x: schaffer_n3(x)[1]},
}

# Executa as otimizações
results = {}

for name, opts in methods.items():
    start_time = time.time()
    res = minimize(lambda x: schaffer_n3(x)[0], x0, **opts, options={"disp": False})
    elapsed_time = (time.time() - start_time) * 1000
    results[name] = {
        "x0": x0,
        "x*": res.x,
        "fval": res.fun,
        "nfev": res.nfev,
        "njev": res.get("njev", None),
        "time": elapsed_time,
        "success": res.success
    }

# Mostra resultados
print("\n==== Schaffer N.3 Results ====\n")
header = f"{'Method':<15} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'Time (ms)':>10} | {'Success':>8}"
print(header)
print("-" * len(header))

for method, data in results.items():
    print(f"{method:<15} | {data['fval']:12.4e} | {data['nfev']:6d} | "
          f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
          f"{data['time']:10.2f} | {str(data['success']):>8}")



==== Schaffer N.3 Results ====

Method          |        f(x*) |   nfev |   njev |  Time (ms) |  Success
------------------------------------------------------------------------
Nelder-Mead     |   1.1573e-12 |    145 |    N/A |       6.01 |     True
BFGS            |   1.3750e-07 |      9 |      9 |       1.00 |     True
CG              |   1.8145e-11 |     13 |     13 |       1.00 |     True
Powell          |   1.0260e-01 |     34 |    N/A |       1.51 |     True
L-BFGS-B        |   3.5568e-08 |      6 |      6 |       0.00 |     True


In [ ]:
## ==== Schaffer N.3 Results ====
## (2.0, -3.0)
## Method          |        f(x*) |   nfev |   njev |  Time (ms) |  Success
## ------------------------------------------------------------------------
## Nelder-Mead     |   6.2242e-03 |    244 |    N/A |       8.05 |     True
## BFGS            |   1.5490e-02 |     65 |     54 |       6.00 |    False
## CG              |   1.3836e-02 |     99 |     89 |       7.26 |    False
## Powell          |   1.3163e-06 |    248 |    N/A |       4.51 |     True
## L-BFGS-B        |   1.2333e-02 |    110 |    110 |       6.00 |    False

## ==== Schaffer N.3 Results ====
## (-10.0, -10.0)
##
## Method          |        f(x*) |   nfev |   njev |  Time (ms) |  Success
## ------------------------------------------------------------------------
## Nelder-Mead     |   3.1266e-03 |    370 |    N/A |       7.51 |     True
## BFGS            |   1.2717e-08 |     10 |     10 |       1.00 |     True
## CG              |   6.1574e-08 |     24 |     24 |       1.00 |     True
## Powell          |   1.5278e-01 |     37 |    N/A |       0.00 |     True
## L-BFGS-B        |   6.8680e-10 |      8 |      8 |       1.00 |     True

## ==== Schaffer N.3 Results ====
## (2.5, -3.5)
##
## Method          |        f(x*) |   nfev |   njev |  Time (ms) |  Success
## ------------------------------------------------------------------------
## Nelder-Mead     |   6.2242e-03 |    254 |    N/A |       5.00 |     True
## BFGS            |   6.2243e-03 |    163 |    152 |      12.01 |    False
## CG              |   8.4878e-03 |    123 |    113 |       8.01 |    False
## Powell          |   2.3626e-02 |     52 |    N/A |       0.00 |     True
## L-BFGS-B        |   6.2242e-03 |     91 |     91 |       3.51 |     True

## ==== Schaffer N.3 Results ====
##  (7.8, -7.8)
##
## Method          |        f(x*) |   nfev |   njev |  Time (ms) |  Success
## ------------------------------------------------------------------------
## Nelder-Mead     |   1.1573e-12 |    145 |    N/A |       6.01 |     True
## BFGS            |   1.3750e-07 |      9 |      9 |       1.00 |     True
## CG              |   1.8145e-11 |     13 |     13 |       1.00 |     True
## Powell          |   1.0260e-01 |     34 |    N/A |       1.51 |     True
## L-BFGS-B        |   3.5568e-08 |      6 |      6 |       0.00 |     True


## Exercício 3 - Função 142 - Streched V Sine Wave Function

## Avaliar para diferentes valores de D

$$
f(x)=Asin(Bx+C)+D
$$

In [31]:
import numpy as np
import time
from scipy.optimize import minimize

# Função Stretched V Sine Wave (com parâmetros A, B, C e D)
def stretched_v_sine_wave(x, A=1, B=1, C=0, D=0):
    fval = A * np.sin(B * x + C) + D

    # Gradiente
    grad = A * B * np.cos(B * x + C)

    return fval, grad

# Ponto inicial (modifique aqui)
x0 = np.array([50.0])

# Valores de D a testar
D_values = [100000]

# Lista de métodos para comparação
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: stretched_v_sine_wave(x, D=0)[1]},
    "CG": {"method": "CG", "jac": lambda x: stretched_v_sine_wave(x, D=0)[1]},
    "Powell": {"method": "Powell"},
    "L-BFGS-B": {"method": "L-BFGS-B", "jac": lambda x: stretched_v_sine_wave(x, D=0)[1]},
}

# Executa as otimizações para diferentes valores de D
results = {}

for D in D_values:
    for name, opts in methods.items():
        start_time = time.time()
        res = minimize(lambda x: stretched_v_sine_wave(x, D=D)[0], x0, **opts, options={"disp": False})
        elapsed_time = (time.time() - start_time) * 1000
        if D not in results:
            results[D] = {}
        results[D][name] = {
            "x0": x0,
            "x*": res.x,
            "fval": res.fun,
            "nfev": res.nfev,
            "njev": res.get("njev", None),
            "time": elapsed_time,
            "success": res.success
        }

# Mostra resultados
for D, result in results.items():
    print(f"\n==== Stretched V Sine Wave Function Results (D={D}) ====\n")
    header = f"{'Method':<15} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'Time (ms)':>10} | {'Success':>8}"
    print(header)
    print("-" * len(header))

    for method, data in result.items():
        print(f"{method:<15} | {data['fval']:12.4e} | {data['nfev']:6d} | "
              f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
              f"{data['time']:10.2f} | {str(data['success']):>8}")



==== Stretched V Sine Wave Function Results (D=100000) ====

Method          |        f(x*) |   nfev |   njev |  Time (ms) |  Success
------------------------------------------------------------------------
Nelder-Mead     |   9.9999e+04 |     34 |    N/A |       2.00 |     True
BFGS            |   9.9999e+04 |      5 |      5 |       2.00 |     True
CG              |   9.9999e+04 |      4 |      4 |       0.51 |     True
Powell          |   9.9999e+04 |     10 |    N/A |       0.00 |     True
L-BFGS-B        |   9.9999e+04 |      5 |      5 |       1.01 |     True


In [ ]:
# ==== Stretched V Sine Wave Function Results (D=300) (x0=75) ====
# 
# Method          |        f(x*) |   nfev |   njev |  Time (ms) |  Success
# ------------------------------------------------------------------------
# Nelder-Mead     |   2.9900e+02 |     37 |    N/A |       2.00 |     True
# BFGS            |   2.9900e+02 |      5 |      5 |       1.00 |     True
# CG              |   2.9900e+02 |      4 |      4 |       0.00 |     True
# Powell          |   2.9900e+02 |     35 |    N/A |       1.00 |     True
# L-BFGS-B        |   2.9900e+02 |      5 |      5 |       1.00 |     True

# ==== Stretched V Sine Wave Function Results (D=5) (x0=10) ====
# 
# Method          |        f(x*) |   nfev |   njev |  Time (ms) |  Success
# ------------------------------------------------------------------------
# Nelder-Mead     |   4.0000e+00 |     30 |    N/A |       1.58 |     True
# BFGS            |   4.0000e+00 |      5 |      5 |       0.53 |     True
# CG              |   4.0000e+00 |      4 |      4 |       0.52 |     True
# Powell          |   4.0000e+00 |     18 |    N/A |       0.52 |     True
# L-BFGS-B        |   4.0000e+00 |      4 |      4 |       0.00 |     True

# ==== Stretched V Sine Wave Function Results (D=20) (x0=1) ====
# 
# Method          |        f(x*) |   nfev |   njev |  Time (ms) |  Success
# ------------------------------------------------------------------------
# Nelder-Mead     |   1.9000e+01 |     38 |    N/A |       0.99 |     True
# BFGS            |   1.9000e+01 |      6 |      6 |       0.00 |     True
# CG              |   1.9000e+01 |     10 |      9 |       1.00 |     True
# Powell          |   1.9000e+01 |     36 |    N/A |       1.00 |     True
# L-BFGS-B        |   1.9000e+01 |      8 |      8 |       0.00 |     True

# ==== Stretched V Sine Wave Function Results (D=100000) (x0=50) ====
# 
# Method          |        f(x*) |   nfev |   njev |  Time (ms) |  Success
# ------------------------------------------------------------------------
# Nelder-Mead     |   9.9999e+04 |     34 |    N/A |       2.00 |     True
# BFGS            |   9.9999e+04 |      5 |      5 |       2.00 |     True
# CG              |   9.9999e+04 |      4 |      4 |       0.51 |     True
# Powell          |   9.9999e+04 |     10 |    N/A |       0.00 |     True
# L-BFGS-B        |   9.9999e+04 |      5 |      5 |       1.01 |     True


## Exercício 4 - Função 61 - Hansen
## A partir de diferentes pontos iniciais, encontrar ao menos dois dos mínimos globais.

$$
f_{61}(\mathbf{x}) = \sum_{i=0}^{4} (i+1)\cos(i x_1 + i + 1)
\sum_{j=0}^{4} (j+1)\cos((j+2)x_2 + j + 1)
$$



In [10]:
import numpy as np
import time
from scipy.optimize import minimize

# Função de Hansen
def hansen(x):
    fval = np.sum([(i+1) * np.cos((i+2)*x + 1) for i in range(len(x))])
    grad = np.array([-(i+1)*(i+2)*np.sin((i+2)*xi + 1) for i, xi in enumerate(x)])
    H = np.diag([(i+1)*(i+2)**2 * np.cos((i+2)*xi + 1) for i, xi in enumerate(x)])
    return fval, grad, H

# Número de dimensões
D = 20  # você pode mudar para qualquer valor

# Ponto inicial manual
x0 = np.array([3.0] * D) # você pode mudar os valores aqui

# Métodos para comparar
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: hansen(x)[1]},
    "CG": {"method": "CG", "jac": lambda x: hansen(x)[1]},
    "L-BFGS-B": {"method": "L-BFGS-B", "jac": lambda x: hansen(x)[1]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: hansen(x)[1], "hess": lambda x: hansen(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: hansen(x)[1], "hess": lambda x: hansen(x)[2]},
}

# Executa as otimizações e armazena resultados
results = {}

for name, opts in methods.items():
    start_time = time.time()
    res = minimize(lambda x: hansen(x)[0], x0, **opts, options={"disp": False})
    elapsed_time = (time.time() - start_time) * 1000
    results[name] = {
        "x0": x0,
        "x*": res.x,
        "fval": res.fun,
        "nfev": res.nfev,
        "njev": res.get("njev", None),
        "nhev": res.get("nhev", None),
        "time": elapsed_time,
        "success": res.success
    }

# Apresenta os resultados
print(f"\n==== Hansen Function Results (D={D}) (x0={x0[0]}) ====\n")
header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
print(header)
print("-" * len(header))

for method, data in results.items():
    print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
          f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
          f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
          f"{data['time']:10.2f} | {str(data['success']):>8}")



==== Hansen Function Results (D=20) (x0=3.0) ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |  -2.0225e+02 |   1451 |    N/A |    N/A |     219.00 |     True
BFGS                 |  -3.6898e+01 |     64 |     52 |    N/A |      19.00 |    False
CG                   |  -1.2185e+01 |     52 |     41 |    N/A |      14.00 |    False
L-BFGS-B             |  -2.4717e+02 |     89 |     89 |    N/A |      24.00 |    False
trust-ncg            |   5.9556e+00 |     30 |      6 |      6 |       7.00 |    False
trust-krylov         |   3.2360e+01 |     29 |     29 |      2 |      20.00 |    False


In [11]:
# ==== Hansen Function Results (D=20) (x0=3) ====

# Resultados para a função Hansen com 20 dimensões e ponto inicial em 3:
# Metodo                |        f(x*)  |   nfev |   njev |   nhev |  Time (ms) |  Success
# --------------------------------------------------------------------------------------
# Nelder-Mead           |  -2.0225e+02  |   1451 |    N/A |    N/A |     211.58 |     True
# BFGS                  |  -3.6898e+01  |     64 |     52 |    N/A |      20.60 |    False
# CG                    |  -1.2185e+01  |     52 |     41 |    N/A |      14.02 |    False
# L-BFGS-B              |  -2.4717e+02  |     89 |     89 |    N/A |      24.81 |    False
# trust-ncg             |   5.9556e+00  |     30 |      6 |      6 |       7.06 |    False
# trust-krylov          |   3.2360e+01  |     29 |     29 |      2 |      20.52 |    False


# ==== Hansen Function Results (D=50) (x0 = np.linspace(-4, 4, 50)) ====

# Resultados para a função Hansen com 50 dimensões e ponto inicial definido por np.linspace(-4, 4, 50):
# Metodo                |        f(x*)  |   nfev |   njev |   nhev |  Time (ms) |  Success
# --------------------------------------------------------------------------------------
# Nelder-Mead           |  -3.9074e+03  |  10000 |    N/A |    N/A |    3483.60 |    False
# BFGS                  |  -1.3094e+03  |     55 |     43 |    N/A |      37.16 |    False
# CG                    |  -1.3094e+03  |     66 |     54 |    N/A |      42.04 |    False
# L-BFGS-B              |  -1.3089e+03  |     43 |     43 |    N/A |      28.13 |    False
# trust-ncg             |  -1.2990e+03  |     30 |      3 |      3 |      13.51 |    False
# trust-krylov          |  -1.3537e+03  |     36 |     36 |      9 |      42.01 |    False


# ==== Hansen Function Results (D=100) (np.random.uniform(-10, 10, 100)) ====

# Resultados para a função Hansen com 100 dimensões e ponto inicial gerado aleatoriamente no intervalo [-10, 10]:
# Metodo                |        f(x*)  |   nfev |   njev |   nhev |  Time (ms) |  Success
# --------------------------------------------------------------------------------------
# Nelder-Mead           |  -2.2625e+04  |  20000 |    N/A |    N/A |   14557.26 |    False
# BFGS                  |  -1.1709e+04  |     59 |     47 |    N/A |      81.13 |    False
# CG                    |  -1.1371e+04  |     46 |     34 |    N/A |      58.83 |    False
# L-BFGS-B              |  -1.2215e+04  |     29 |     29 |    N/A |      40.60 |     True
# trust-ncg             |  -1.2186e+04  |     34 |      4 |      4 |      39.26 |    False
# trust-krylov          |  -1.1743e+04  |     39 |     39 |     13 |      89.00 |    False


# ==== Hansen Function Results (D=50) (x0= np.full(D, 0.0)) ====

# Resultados para a função Hansen com 50 dimensões e ponto inicial em vetor de zeros:
# Método em que o ponto inicial está em vetor de zeros, e seria suposto ser fácil de encontrar o mínimo.
# Metodo                |        f(x*)  |   nfev |   njev |   nhev |  Time (ms) |  Success
# --------------------------------------------------------------------------------------
# Nelder-Mead           |  -3.4896e+04  |  10000 |    N/A |    N/A |    3393.77 |    False
# BFGS                  |  -6.4597e+03  |     59 |     47 |    N/A |      38.25 |    False
# CG                    |  -7.6987e+03  |     68 |     57 |    N/A |      44.12 |    False
# L-BFGS-B              |  -1.8924e+04  |    115 |    115 |    N/A |      75.02 |    False
# trust-ncg             |  -1.2546e+04  |     39 |     12 |     12 |      22.10 |    False
# trust-krylov          |  -2.7779e+04  |     64 |     64 |     45 |      86.04 |    False


# ==== Hansen Function Results (D=100) (x0=0.001 * np.random.randn(D)) ====

# Resultados para a função Hansen com 100 dimensões e ponto inicial com pequenas perturbações em torno de zero:
# Metodo                |        f(x*)  |   nfev |   njev |   nhev |  Time (ms) |  Success
# --------------------------------------------------------------------------------------
# Nelder-Mead           |  -1.0156e+05  |  20000 |    N/A |    N/A |   13598.07 |    False
# BFGS                  |  -3.3912e+04  |     50 |     39 |    N/A |      62.71 |    False
# CG                    |  -5.3291e+04  |     65 |     53 |    N/A |      82.71 |    False
# L-BFGS-B              |  -1.2706e+05  |     95 |     95 |    N/A |     125.19 |     True
# trust-ncg             |  -2.8040e+04  |     35 |      6 |      6 |      40.15 |    False
# trust-krylov          |  -2.0069e+05  |     62 |     62 |     45 |     154.32 |    False


## Exercício 5.1

$$
\min_{\mathbf{x} \in \mathbb{R}^2} \; x_1^2 + x_2^2
$$

In [25]:
import sympy as sp

# Definindo as variáveis
x1, x2 = sp.symbols('x1 x2')

# Definindo a função objetivo
f = x1**2 + x2**2

# 1. Calculando o gradiente
grad_f = [sp.diff(f, var) for var in (x1, x2)]
print("Gradiente:")
print(grad_f)

# 2. Calculando a matriz Hessiana
hess_f = sp.hessian(f, (x1, x2))
print("\nHessiana:")
sp.pprint(hess_f)

# 3. Encontrando os pontos críticos (gradiente = 0)
critical_points = sp.solve(grad_f, (x1, x2), dict=True)
print("\nPontos críticos:")
print(critical_points)

# 4. Verificando as condições de segunda ordem (autovalores da Hessiana)
print("\nAnálise dos pontos críticos:")
for point_dict in critical_points:
    hess_at_point = hess_f.subs(point_dict)
    eigenvals = hess_at_point.eigenvals()

    point = (point_dict[x1], point_dict[x2])
    print(f"\nPonto crítico: {point}")
    print("Hessiana no ponto:")
    sp.pprint(hess_at_point)
    print("Autovalores:", eigenvals)

    # Classificação
    eigenvalues_list = list(eigenvals.keys())
    if all(val > 0 for val in eigenvalues_list):
        print("→ Mínimo local")
    elif all(val < 0 for val in eigenvalues_list):
        print("→ Máximo local")
    else:
        print("→ Ponto de sela")




Gradiente:
[2*x1, 2*x2]

Hessiana:
⎡2  0⎤
⎢    ⎥
⎣0  2⎦

Pontos críticos:
[{x1: 0, x2: 0}]

Análise dos pontos críticos:

Ponto crítico: (0, 0)
Hessiana no ponto:
⎡2  0⎤
⎢    ⎥
⎣0  2⎦
Autovalores: {2: 2}
→ Mínimo local


In [ ]:
# Gradiente:
# [2*x1, 2*x2]

# Hessiana:
# ⎡2  0⎤
# ⎢    ⎥
# ⎣0  2⎦

# Pontos críticos:
# [{x1: 0, x2: 0}]

# Análise dos pontos críticos:

# Ponto crítico: (0, 0)
# Hessiana no ponto:
# ⎡2  0⎤
# ⎢    ⎥
# ⎣0  2⎦
# Autovalores: {2: 2}
# → Mínimo local


## Exercício 5.2

$$
 \min_{\mathbf{x} \in \mathbb{R}^2} \; \frac{1}{3}x_1^3 + x_2^3 - x_1 - x_2
$$

In [20]:
import sympy as sp

# Definindo as variáveis
x1, x2 = sp.symbols('x1 x2')

# Definindo a função objetivo
f = (1 / 3) * x1 ** 3 + x2 ** 3 - x1 - x2

# 1. Calculando o gradiente
grad_f = [sp.diff(f, var) for var in (x1, x2)]
print("Gradiente:")
print(grad_f)

# 2. Calculando a matriz Hessiana
hess_f = sp.hessian(f, (x1, x2))
print("\nHessiana:")
sp.pprint(hess_f)

# 3. Encontrando os pontos críticos (gradiente = 0)
critical_points = sp.solve(grad_f, (x1, x2))
print("\nPontos críticos:")
print(critical_points)

# 4. Verificando as condições de segunda ordem (autovalores da Hessiana)
print("\nAnálise dos pontos críticos:")
for point in critical_points:
    hess_at_point = hess_f.subs({x1: point[0], x2: point[1]})
    eigenvals = hess_at_point.eigenvals()

    print(f"\nPonto crítico: {point}")
    print("Hessiana no ponto:")
    sp.pprint(hess_at_point)
    print("Autovalores:", eigenvals)

    # Classificação
    eigenvalues_list = list(eigenvals.keys())
    if all(val > 0 for val in eigenvalues_list):
        print("→ Mínimo local")
    elif all(val < 0 for val in eigenvalues_list):
        print("→ Máximo local")
    else:
        print("→ Ponto de sela")


Gradiente:
[1.0*x1**2 - 1, 3*x2**2 - 1]

Hessiana:
⎡2.0⋅x₁   0  ⎤
⎢            ⎥
⎣  0     6⋅x₂⎦

Pontos críticos:
[(-1.00000000000000, -0.577350269189626), (-1.00000000000000, 0.577350269189626), (1.00000000000000, -0.577350269189626), (1.00000000000000, 0.577350269189626)]

Análise dos pontos críticos:

Ponto crítico: (-1.00000000000000, -0.577350269189626)
Hessiana no ponto:
⎡-2.0          0        ⎤
⎢                       ⎥
⎣ 0    -3.46410161513775⎦
Autovalores: {-2.00000000000000: 1, -3.46410161513775: 1}
→ Máximo local

Ponto crítico: (-1.00000000000000, 0.577350269189626)
Hessiana no ponto:
⎡-2.0         0        ⎤
⎢                      ⎥
⎣ 0    3.46410161513775⎦
Autovalores: {-2.00000000000000: 1, 3.46410161513775: 1}
→ Ponto de sela

Ponto crítico: (1.00000000000000, -0.577350269189626)
Hessiana no ponto:
⎡2.0          0        ⎤
⎢                      ⎥
⎣ 0   -3.46410161513775⎦
Autovalores: {2.00000000000000: 1, -3.46410161513775: 1}
→ Ponto de sela

Ponto crítico: (1.000000

In [ ]:
# Gradiente da função:
# [1.0 * x1**2 - 1, 3 * x2**2 - 1]

# Hessiana da função:
# [ [ 2.0 * x1     ,     0           ],
#   [    0         ,   6.0 * x2      ] ]

# Pontos críticos (soluções do sistema gradiente = 0):
# (-1.00000000000000, -0.577350269189626)
# (-1.00000000000000,  0.577350269189626)
# ( 1.00000000000000, -0.577350269189626)
# ( 1.00000000000000,  0.577350269189626)

# Análise dos pontos críticos:

# Ponto crítico: (-1.0, -0.577350269189626)
# Hessiana:
# [ [-2.0               ,     0                  ],
#   [  0                ,   -3.46410161513775    ] ]
# Autovalores: {-2.0: 1, -3.46410161513775: 1}
# → Máximo local

# Ponto crítico: (-1.0, 0.577350269189626)
# Hessiana:
# [ [-2.0               ,     0                  ],
#   [  0                ,    3.46410161513775    ] ]
# Autovalores: {-2.0: 1, 3.46410161513775: 1}
# → Ponto de sela

# Ponto crítico: (1.0, -0.577350269189626)
# Hessiana:
# [ [ 2.0               ,     0                  ],
#   [  0                ,   -3.46410161513775    ] ]
# Autovalores: {2.0: 1, -3.46410161513775: 1}
# → Ponto de sela

# Ponto crítico: (1.0, 0.577350269189626)
# Hessiana:
# [ [ 2.0               ,     0                  ],
#   [  0                ,    3.46410161513775    ] ]
# Autovalores: {2.0: 1, 3.46410161513775: 1}
# → Mínimo local


## Exercício 5.3

$$
\min_{x \in \mathbb{R}} \; x^2 + \frac{1}{x - 2}
$$

In [21]:
import sympy as sp
from scipy.optimize import minimize_scalar

# 1. Definindo a variável simbólica
x = sp.Symbol('x')
f = x ** 2 + 1 / (x - 2)

# 2. Derivada (gradiente)
df = sp.diff(f, x)
print("Gradiente (f'):")
sp.pprint(df)

# 3. Segunda derivada (Hessiana em R¹)
d2f = sp.diff(df, x)
print("\nSegunda derivada (f''):")
sp.pprint(d2f)

# 4. Tentando encontrar pontos críticos reais (df = 0)
critical_points = sp.solve(df, x)

# 5. Filtrar apenas os pontos reais e diferentes de x = 2
valid_points = [
    pt for pt in critical_points
    if pt.is_real and not sp.simplify(pt - 2) == 0
]

print("\nPontos críticos reais (válidos):")
print(valid_points)

# 6. Análise da segunda derivada nos pontos críticos
for pt in valid_points:
    second_derivative_value = d2f.subs(x, pt)
    print(f"\nNo ponto x = {pt}: f''(x) = {second_derivative_value}")
    if second_derivative_value > 0:
        print("→ Mínimo local")
    elif second_derivative_value < 0:
        print("→ Máximo local")
    else:
        print("→ Ponto de inflexão")

# 7. Caso não haja ponto crítico real, encontrar mínimo numérico
if not valid_points:
    print("\nNão há ponto crítico real viável. Fazendo busca numérica no intervalo (2.01, 5)...")


    def f_num(x_val):
        return x_val ** 2 + 1 / (x_val - 2)


    result = minimize_scalar(f_num, bounds=(2.01, 5), method='bounded')

    print(f"\nMínimo numérico aproximado:")
    print(f"x ≈ {result.x:.4f}")
    print(f"f(x) ≈ {result.fun:.4f}")

Gradiente (f'):
         1    
2⋅x - ────────
             2
      (x - 2) 

Segunda derivada (f''):
       2    
2 + ────────
           3
    (x - 2) 

Pontos críticos reais (válidos):
[]

Não há ponto crítico real viável. Fazendo busca numérica no intervalo (2.01, 5)...

Mínimo numérico aproximado:
x ≈ 2.4516
f(x) ≈ 8.2247


In [ ]:
# Gradiente (f'):
#          1    
# 2⋅x - ────────
#              2
#       (x - 2) 

# Segunda derivada (f''):
#        2    
# 2 + ────────
#            3
#     (x - 2) 

# Pontos críticos reais (válidos):
# []

# Não há ponto crítico real viável. Fazendo busca numérica no intervalo (2.01, 5)...

# Mínimo numérico aproximado:
# x ≈ 2.4516
# f(x) ≈ 8.2247


## Exercício 5.4

$$
 \min_{x \in \mathbb{R}^2} \; x_1^6 - 3x_1^4 x_2^2 + 3x_1^2 x_2^4 - x_2^6
$$


In [26]:
import sympy as sp

# 1. Definindo as variáveis simbólicas
x1, x2 = sp.symbols('x1 x2')
f = x1**6 - 3*x1**4*x2**2 + 3*x1**2*x2**4 - x2**6

# 2. Gradiente (primeiras derivadas)
grad_f = [sp.diff(f, var) for var in (x1, x2)]
print("Gradiente:")
print(grad_f)

# 3. Hessiana (matriz de segundas derivadas)
hess_f = sp.hessian(f, (x1, x2))
print("\nHessiana:")
sp.pprint(hess_f)

# 4. Pontos críticos (resolvendo gradiente = 0)
critical_points = sp.solve(grad_f, (x1, x2), dict=True)
valid_points = [pt for pt in critical_points if x1 in pt and x2 in pt]

print("\nPontos críticos encontrados:")
print(valid_points)

# 5. Avaliação da Hessiana e autovalores
for pt in valid_points:
    point_val = (pt[x1], pt[x2])
    hess_at_point = hess_f.subs(pt)
    eigenvals = hess_at_point.eigenvals()

    print(f"\nAnálise no ponto crítico: {point_val}")
    print("Hessiana avaliada no ponto:")
    sp.pprint(hess_at_point)
    print("Autovalores da Hessiana:", eigenvals)

    # Classificação
    eigenvalues_list = list(eigenvals.keys())
    if all(val > 0 for val in eigenvalues_list):
        print("→ Mínimo local")
    elif all(val < 0 for val in eigenvalues_list):
        print("→ Máximo local")
    elif all(val == 0 for val in eigenvalues_list):
        print("→ Condição de segunda ordem inconclusiva (todos os autovalores são zero)")
    else:
        print("→ Ponto de sela ou condição inconclusiva")


Gradiente:
[6*x1**5 - 12*x1**3*x2**2 + 6*x1*x2**4, -6*x1**4*x2 + 12*x1**2*x2**3 - 6*x2**5]

Hessiana:
⎡     4        2   2       4            3              3   ⎤
⎢30⋅x₁  - 36⋅x₁ ⋅x₂  + 6⋅x₂      - 24⋅x₁ ⋅x₂ + 24⋅x₁⋅x₂    ⎥
⎢                                                          ⎥
⎢         3              3          4        2   2        4⎥
⎣  - 24⋅x₁ ⋅x₂ + 24⋅x₁⋅x₂     - 6⋅x₁  + 36⋅x₁ ⋅x₂  - 30⋅x₂ ⎦

Pontos críticos encontrados:
[{x1: 0, x2: 0}]

Análise no ponto crítico: (0, 0)
Hessiana avaliada no ponto:
⎡0  0⎤
⎢    ⎥
⎣0  0⎦
Autovalores da Hessiana: {0: 2}
→ Condição de segunda ordem inconclusiva (todos os autovalores são zero)


In [ ]:
# Gradiente:
# [6*x1**5 - 12*x1**3*x2**2 + 6*x1*x2**4, -6*x1**4*x2 + 12*x1**2*x2**3 - 6*x2**5]

# Hessiana:
# ⎡     4        2   2       4            3              3   ⎤
# ⎢30⋅x₁  - 36⋅x₁ ⋅x₂  + 6⋅x₂      - 24⋅x₁ ⋅x₂ + 24⋅x₁⋅x₂    ⎥
# ⎢                                                          ⎥
# ⎢         3              3          4        2   2        4⎥
# ⎣  - 24⋅x₁ ⋅x₂ + 24⋅x₁⋅x₂     - 6⋅x₁  + 36⋅x₁ ⋅x₂  - 30⋅x₂ ⎦

# Pontos críticos encontrados:
# [{x1: 0, x2: 0}]

# Análise no ponto crítico: (0, 0)
# Hessiana avaliada no ponto:
# ⎡0  0⎤
# ⎢    ⎥
# ⎣0  0⎦

# Autovalores da Hessiana: {0: 2}
# → Condição de segunda ordem inconclusiva (todos os autovalores são zero)
